### Configuración del entorno y verificación de GPU

En primer lugar se configura el entorno de ejecución para que TensorFlow utilice la GPU de forma segura, evitando errores en ciertas operaciones de convolución y permitiendo que la memoria de la GPU crezca de manera dinámica según la demanda. Luego se verifica la disponibilidad y características de la GPU mediante el comando `nvidia-smi`.


In [ ]:
import os
os.environ["XLA_FLAGS"] = "--xla_gpu_strict_conv_algorithm_picker=false"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Nov 21 00:07:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             51W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

### Instalación de librerías adicionales

Se instalan las librerías necesarias para el manejo de imágenes geoespaciales (`rasterio`) y para la definición de arquitecturas de segmentación profundas (`segmentation-models`). Estas librerías no vienen preinstaladas en Colab, por lo que es necesario incorporarlas explícitamente antes de construir y entrenar el modelo.


In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 124.4 MB/s eta 0:00:00


In [ ]:
!pip install -U segmentation-models==1.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.4 MB/s eta 0:00:00


### Selección de backend y acceso a Google Drive

Se configura la librería `segmentation-models` para que utilice `tf.keras` como backend. Además, se monta Google Drive con el fin de:
1. Cargar el dataset de entrenamiento/validación/test (almacenado previamente en Drive).
2. Guardar los modelos entrenados y otros artefactos (checkpoints, logs) de forma persistente.


In [ ]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
os.environ["SM_BACKEND"] = "tensorflow"

In [ ]:
import segmentation_models as sm
sm.set_framework('tf.keras')

Segmentation Models: using `tf.keras` framework.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Carga y estructura del dataset de parches

El dataset de entrada se encuentra comprimido en un archivo `.zip` en Google Drive. Primero se descomprime en el entorno de Colab y se define la ruta raíz del dataset (`DATASET_ROOT`), que contiene tres subcarpetas: `train`, `val` y `test`.

Luego se implementa la función `get_image_mask_paths(split)` que, para un subconjunto dado (`train`, `val` o `test`), genera dos listas paralelas:
- Rutas de las imágenes satelitales (parches GeoTIFF).
- Rutas de las máscaras de segmentación correspondientes, construidas a partir del nombre de cada imagen.


In [ ]:
import os, glob, math
import numpy as np
import tensorflow as tf
import rasterio
import cv2

# ====== DESCOMPRIMIR ZIP EN COLAB ======
!unzip -q "/content/drive/MyDrive/Proyecto Integrador/dataset_final.zip" -d "/content"

DATASET_ROOT = "/content/dataset_final"  # carpeta que contiene train/valid/test

print("Subcarpetas:", os.listdir(DATASET_ROOT))

Subcarpetas: ['val', 'test', 'train']


In [ ]:
def get_image_mask_paths(split):
    img_dir = os.path.join(DATASET_ROOT, split, "images")
    mask_dir = os.path.join(DATASET_ROOT, split, "masks")

    img_paths = sorted(glob.glob(os.path.join(img_dir, "*.tif")))
    mask_paths = []

    for p in img_paths:
        fname = os.path.basename(p)  # img_000001.tif
        mask_name = fname.replace("img_", "mask_")
        mask_paths.append(os.path.join(mask_dir, mask_name))

    print(f"{split}: {len(img_paths)} imágenes")
    return img_paths, mask_paths

train_img_paths, train_mask_paths = get_image_mask_paths("train")
val_img_paths,   val_mask_paths   = get_image_mask_paths("val")

train: 2467 imágenes
val: 803 imágenes


### Generador de datos para archivos GeoTIFF

Para alimentar el modelo se define la clase `TIFFDataGenerator`, basada en `tf.keras.utils.Sequence`. Este generador:

1. Recibe listas de rutas de imágenes y máscaras en formato GeoTIFF.
2. Lee cada imagen con `rasterio` y la transforma al formato `(alto, ancho, canales)`, forzando a trabajar siempre con 3 canales (RGB).
3. Redimensiona imágenes y máscaras al tamaño objetivo (512×512 píxeles).
4. Normaliza cada parche de imagen mediante una escala min–max, llevando los valores al rango [0, 1].
5. Construye la máscara en formato one-hot (tres clases: fondo, área urbana y área rural) y genera una máscara binaria de pesos (`valid_mask`) que permite ignorar los píxeles marcados con `ignore_index` (255).
6. Devuelve, para cada batch, un triplete `(X, y, w)` compuesto por imágenes normalizadas, máscaras one-hot y pesos de muestra, listo para ser utilizado por el modelo durante el entrenamiento y la evaluación.


In [ ]:
class TIFFDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, mask_paths,
                 batch_size=4,
                 shuffle=True,
                 normalize=True,
                 target_size=(512, 512),
                 n_channels=3,      # <<< RGB
                 n_classes=3,
                 ignore_index=255):

        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.normalize = normalize
        self.target_size = target_size
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.ignore_index = ignore_index
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.image_paths) / self.batch_size)

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_imgs, batch_masks, batch_weights = [], [], []

        for i in batch_indexes:

            # ---------- Leer imagen ----------
            with rasterio.open(self.image_paths[i]) as src:
                img = src.read()               # (C, H, W)
                img = np.transpose(img, (1, 2, 0))  # (H, W, C)

            # ---------- Leer máscara ----------
            with rasterio.open(self.mask_paths[i]) as src:
                mask = src.read(1)

            img = img.astype(np.float32)
            mask = mask.astype(np.int32)

            # ---------- Ajustar a 3 canales RGB ----------
            if img.shape[-1] > 3:
                img = img[..., :3]
            elif img.shape[-1] < 3:
                while img.shape[-1] < 3:
                    img = np.concatenate([img, img[..., -1:]], axis=-1)

            # ---------- Redimensionar ----------
            if img.shape[0:2] != self.target_size:
                img = cv2.resize(img, self.target_size, interpolation=cv2.INTER_LINEAR)
            if mask.shape[0:2] != self.target_size:
                mask = cv2.resize(mask, self.target_size, interpolation=cv2.INTER_NEAREST)

            # ---------- Normalización ----------
            if self.normalize:
                img_min, img_max = img.min(), img.max()
                if img_max > img_min:
                    img = (img - img_min) / (img_max - img_min)
                else:
                    img = np.zeros_like(img)

            # ---------- sample_weights ----------
            valid_mask = (mask != self.ignore_index).astype("float32")

            # ---------- Limitar mask a [0,1,2] ----------
            mask_clipped = np.clip(mask, 0, self.n_classes - 1)

            # ---------- One hot ----------
            one_hot = np.eye(self.n_classes, dtype="float32")[mask_clipped]

            batch_imgs.append(img)
            batch_masks.append(one_hot)
            batch_weights.append(valid_mask[..., None])

        X = np.stack(batch_imgs, axis=0)
        y = np.stack(batch_masks, axis=0)
        w = np.stack(batch_weights, axis=0)

        return X, y, w

### Definición de generadores de entrenamiento y validación

Utilizando la clase `TIFFDataGenerator`, se construyen dos generadores:
- `train_gen`, que baraja los parches en cada época (`shuffle=True`).
- `val_gen`, que mantiene un orden fijo (`shuffle=False`) para evaluar de forma consistente.

Ambos generadores producen batches de tamaño 4, con imágenes RGB de 512×512 y máscaras one-hot de tres clases (fondo, urbano, rural).


In [ ]:
train_gen = TIFFDataGenerator(train_img_paths, train_mask_paths,
                              batch_size=4, shuffle=True,
                              n_channels=3, n_classes=3, ignore_index=255)

val_gen = TIFFDataGenerator(val_img_paths, val_mask_paths,
                            batch_size=4, shuffle=False,
                            n_channels=3, n_classes=3, ignore_index=255)


### Modelo de segmentación: U-Net con backbone ResNet34

Para la tarea de segmentación semántica multiclase se utiliza la arquitectura U-Net implementada en la librería `segmentation-models`, empleando como encoder un backbone `ResNet34` preentrenado en ImageNet. El modelo trabaja con entradas RGB de 512×512 píxeles y produce, para cada píxel, una distribución de probabilidad sobre tres clases (fondo, urbano y rural) mediante una capa de salida con activación `softmax`.


In [ ]:
from tensorflow.keras.optimizers import Adam

BACKBONE = 'resnet34'     # igual que tu amigo
N_CLASSES = 3             # background, urbano, rural

# ------------------------------
# UNet + ResNet34 + ImageNet
# ------------------------------
model = sm.Unet(
    backbone_name=BACKBONE,
    encoder_weights='imagenet',           # <<< PREENTRENADO RGB
    classes=N_CLASSES,
    activation='softmax',                 # multiclase
    input_shape=(512, 512, 3)             # <<< RGB
)

### Función de pérdida y métricas de evaluación

Se definen métricas personalizadas de IoU (Intersection over Union) y F1 por clase (`fondo`, `urbano`, `rural`) a partir de las salidas one-hot del modelo. Además, se emplea una función de pérdida compuesta por:

- `CategoricalFocalLoss`: que penaliza más los ejemplos difíciles y ayuda a manejar el desbalance entre clases.
- `JaccardLoss` (basada en IoU): que optimiza directamente la superposición entre predicción y verdad terreno.

El optimizador utilizado es Adam con una tasa de aprendizaje inicial de `1e-4`, y se monitorizan tanto las métricas globales (`iou_score`, `f1_score`) como las métricas por clase.


In [ ]:
# -------------------- Métricas personalizadas por clase --------------------
import tensorflow as tf

# Métrica IoU para una clase específica
def iou_for_class(class_id):
    def metric(y_true, y_pred):
        y_true_c = tf.cast(tf.equal(tf.argmax(y_true, -1), class_id), tf.float32)
        y_pred_c = tf.cast(tf.equal(tf.argmax(y_pred, -1), class_id), tf.float32)
        inter = tf.reduce_sum(y_true_c * y_pred_c)
        union = tf.reduce_sum(y_true_c) + tf.reduce_sum(y_pred_c) - inter + 1e-7
        return inter / union
    metric.__name__ = f'iou_class_{class_id}'
    return metric

# Métrica F1 para una clase
def f1_for_class(class_id):
    def metric(y_true, y_pred):
        y_true_c = tf.cast(tf.equal(tf.argmax(y_true, -1), class_id), tf.float32)
        y_pred_c = tf.cast(tf.equal(tf.argmax(y_pred, -1), class_id), tf.float32)
        tp = tf.reduce_sum(y_true_c * y_pred_c)
        fp = tf.reduce_sum((1 - y_true_c) * y_pred_c)
        fn = tf.reduce_sum(y_true_c * (1 - y_pred_c))
        precision = tp / (tp + fp + 1e-7)
        recall    = tp / (tp + fn + 1e-7)
        return 2 * precision * recall / (precision + recall + 1e-7)
    metric.__name__ = f'f1_class_{class_id}'
    return metric

# -------------------- Pérdida y métricas globales --------------------
loss = sm.losses.CategoricalFocalLoss() + sm.losses.JaccardLoss()

metrics = [
    sm.metrics.IOUScore(threshold=None, name="iou_score"),
    sm.metrics.FScore(threshold=None, name="f1_score"),
    iou_for_class(0),  # background
    iou_for_class(1),  # urbano
    iou_for_class(2),  # rural
    f1_for_class(0),
    f1_for_class(1),
    f1_for_class(2),
]

In [ ]:
model.compile(
    optimizer=Adam(1e-4),
    loss=loss,
    metrics=[
        sm.metrics.IOUScore(threshold=None, name="iou_score"),
        sm.metrics.FScore(threshold=None, name="f1_score"),

        iou_for_class(0),
        iou_for_class(1),
        iou_for_class(2),

        f1_for_class(0),
        f1_for_class(1),
        f1_for_class(2),
    ]
)

model.summary()


### Callbacks: parada temprana y guardado de modelos

Para controlar el entrenamiento se emplean varios callbacks:

- **EarlyStopping**: detiene el entrenamiento cuando la métrica `val_iou_score` deja de mejorar durante un número determinado de épocas, restaurando los mejores pesos observados.
- **ModelCheckpoint (mejor modelo)**: guarda en Google Drive el modelo con el mayor `val_iou_score`.
- **ModelCheckpoint (por época)**: almacena un checkpoint adicional por cada época, permitiendo revisar versiones intermedias del modelo.
- **TensorBoard**: registra los logs de entrenamiento y validación para su posterior visualización gráfica (pérdidas, métricas, etc.).


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

CHECKPOINT_DIR = "/content/drive/MyDrive/Proyecto Integrador/ccpp_checkpoints"

os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs("logs", exist_ok=True)

early_stop = EarlyStopping(
    monitor='val_iou_score',
    mode='max',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

checkpoint_best = ModelCheckpoint(
    filepath=os.path.join(CHECKPOINT_DIR, "best_unet_multiclass.keras"),
    monitor='val_iou_score',
    mode='max',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

checkpoint_every = ModelCheckpoint(
    filepath="checkpoints/ckpt_epoch_{epoch:02d}.keras",
    save_best_only=False,
    save_weights_only=False,
    verbose=0
)

tensorboard_cb = TensorBoard(
    log_dir="logs/unet_multiclass",
    histogram_freq=0
)


### Entrenamiento del modelo

El modelo se entrena utilizando los generadores de entrenamiento y validación durante un máximo de 60 épocas. El proceso está controlado por la parada temprana, de modo que el entrenamiento se detiene automáticamente cuando la métrica `val_iou_score` deja de mejorar, evitando sobreajuste. Al finalizar, se dispone del mejor modelo según el desempeño en el conjunto de validación.


In [ ]:
EPOCHS = 60

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[early_stop, checkpoint_best, checkpoint_every, tensorboard_cb]
)

print("Epochs realmente entrenadas:", len(history.history['loss']))


Epoch 1/60
617/617 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - f1_class_0: 0.9071 - f1_class_1: 0.0559 - f1_class_2: 0.3150 - f1_score: 0.3148 - iou_class_0: 0.8546 - iou_class_1: 0.0426 - iou_class_2: 0.2084 - iou_score: 0.2503 - loss: 0.7703
Epoch 1: val_iou_score improved from -inf to 0.30866, saving model to /content/drive/MyDrive/Proyecto Integrador/ccpp_checkpoints/best_unet_multiclass.keras
617/617 ━━━━━━━━━━━━━━━━━━━━ 154s 163ms/step - f1_class_0: 0.9072 - f1_class_1: 0.0560 - f1_class_2: 0.3152 - f1_score: 0.3149 - iou_class_0: 0.8547 - iou_class_1: 0.0426 - iou_class_2: 0.2086 - iou_score: 0.2505 - loss: 0.7701 - val_f1_class_0: 0.9661 - val_f1_class_1: 0.0000e+00 - val_f1_class_2: 0.0000e+00 - val_f1_score: 0.3226 - val_iou_class_0: 0.9439 - val_iou_class_1: 0.0000e+00 - val_iou_class_2: 0.0000e+00 - val_iou_score: 0.3087 - val_loss: 0.7105
Epoch 2/60
617/617 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - f1_class_0: 0.9867 - f1_class_1: 0.1660 - f1_class_2: 0.5806 - f1_score: 0.5486 - iou_cl

### Recarga y afinamiento adicional del mejor modelo

En caso de ser necesario, se recarga explícitamente el mejor modelo guardado mediante `ModelCheckpoint`, especificando las funciones de pérdida y métricas personalizadas (`custom_objects`). A partir de este punto es posible realizar un afinamiento adicional (fine-tuning) sobre el mismo conjunto de entrenamiento y validación, manteniendo los mismos callbacks y criterios de parada.


In [ ]:
import segmentation_models as sm
import tensorflow as tf
import os

os.environ["SM_FRAMEWORK"] = "tf.keras"
sm.set_framework('tf.keras')

custom_objects = {
    'CategoricalFocalJaccardLoss': sm.losses.CategoricalFocalJaccardLoss,
    'iou_score': sm.metrics.IOUScore,
    'f1_score': sm.metrics.FScore,
    # métricas custom:
    'iou_class_0': iou_for_class(0),
    'iou_class_1': iou_for_class(1),
    'iou_class_2': iou_for_class(2),
    'f1_class_0':  f1_for_class(0),
    'f1_class_1':  f1_for_class(1),
    'f1_class_2':  f1_for_class(2),
}

ckpt_path = "checkpoints/best_unet_multiclass.keras"  # o uno de los ckpt_epoch_XX
assert os.path.exists(ckpt_path), "No existe el checkpoint elegido"

model = tf.keras.models.load_model(ckpt_path, custom_objects=custom_objects)
print("Modelo cargado desde:", ckpt_path)


In [ ]:
history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,   # epochs adicionales
    callbacks=[early_stop, checkpoint_best, checkpoint_every, tensorboard_cb]
)


### Evaluación en el conjunto de prueba

Para medir el desempeño final del modelo se construye un generador específico para el conjunto de prueba (`test_gen`), que recorre todas las imágenes de test con `batch_size=1`. A continuación, se utiliza `model.evaluate` para obtener la pérdida y las métricas definidas (IoU y F1 globales y por clase), proporcionando una estimación objetiva de la capacidad de generalización del modelo sobre parches no vistos durante el entrenamiento ni la validación.


In [ ]:
import glob
import os

TEST_IMG_DIR = os.path.join(DATASET_ROOT, "test/images")
TEST_MASK_DIR = os.path.join(DATASET_ROOT, "test/masks")

test_img_paths = sorted(glob.glob(os.path.join(TEST_IMG_DIR, "*.tif")))
test_mask_paths = sorted(glob.glob(os.path.join(TEST_MASK_DIR, "*.tif")))

print("Test imágenes:", len(test_img_paths))
print("Test máscaras:", len(test_mask_paths))

Test imágenes: 479
Test máscaras: 479


In [ ]:
test_gen = TIFFDataGenerator(
    test_img_paths,
    test_mask_paths,
    batch_size=1,
    shuffle=False,
    target_size=(512,512),
    n_channels=3,
    n_classes=3
)

In [ ]:
test_metrics = model.evaluate(test_gen, verbose=1)
for name, value in zip(model.metrics_names, test_metrics):
    print(f"{name}: {value}")

479/479 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - f1_class_0: 0.9970 - f1_class_1: 0.0000e+00 - f1_class_2: 0.5994 - f1_score: 0.5316 - iou_class_0: 0.9940 - iou_class_1: 0.0000e+00 - iou_class_2: 0.4728 - iou_score: 0.4883 - loss: 0.5165
loss: 0.5199595093727112
compile_metrics: 0.48464134335517883


### Predicción preliminar (sanity check) sobre un parche de test

Antes de generar y guardar las predicciones completas, se realiza una prueba rápida
(sanity check) tomando el primer batch del conjunto de prueba. Esto permite verificar
que:
- El generador produce imágenes y máscaras correctamente procesadas.
- El modelo devuelve probabilidades softmax con forma `(512,512,3)`.
- La conversión a clases discretas mediante `argmax` produce una máscara de clases
de forma `(512,512)`.

Esta verificación asegura que todo el pipeline de inferencia funciona correctamente
antes de procesar el conjunto completo.


In [ ]:
X, y_true, w = test_gen[0]
y_pred = model.predict(X)

pred_softmax = y_pred[0]      # (512,512,3)
pred_class = pred_softmax.argmax(axis=-1)   # (512,512)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


### Generación y almacenamiento de predicciones por parche

Una vez entrenado el modelo, se generan predicciones para cada parche del conjunto de prueba. Para cada imagen:

1. Se lee el GeoTIFF original y se conserva su metadata geoespacial (CRS, transformada y dimensiones).
2. Se normaliza la imagen de la misma forma que en el generador de datos.
3. Se obtiene la predicción del modelo y se asigna a cada píxel la clase de mayor probabilidad (`argmax`).
4. Se guarda la máscara de clases resultante como un nuevo GeoTIFF (uint8), reutilizando la metadata original, de modo que cada predicción mantiene la georreferenciación del parche de entrada.

Estas salidas se almacenan en la carpeta `/content/predictions`.


In [ ]:
import rasterio
import numpy as np
import os

os.makedirs("/content/predictions", exist_ok=True)

for i, img_path in enumerate(test_img_paths):

    # --- Leer imagen original ---
    with rasterio.open(img_path) as src:
        meta = src.meta.copy()
        img = src.read()               # (C,H,W)
        img = np.transpose(img, (1,2,0)).astype(np.float32)

    # --- Forzar RGB ---
    img = img[..., :3]

    # --- Normalizar ---
    img = (img - img.min()) / (img.max() - img.min() + 1e-6)

    # --- Expandir batch ---
    X = np.expand_dims(img, axis=0)

    # --- Predecir ---
    pred_softmax = model.predict(X, verbose=0)[0]
    pred_class = pred_softmax.argmax(axis=-1).astype("uint8")  # (512,512)

    # --- Guardar como GeoTIFF ---
    meta.update({
        "count": 1,
        "dtype": "uint8"
    })

    out_path = f"/content/predictions/pred_{i:05d}.tif"

    with rasterio.open(out_path, "w", **meta) as dst:
        dst.write(pred_class, 1)

print("Listo: todas las predicciones guardadas en /content/predictions")

Listo: todas las predicciones guardadas en /content/predictions


### Reconstrucción del mosaico georreferenciado de predicciones

Finalmente, todas las máscaras de predicción individuales (un GeoTIFF por parche) se combinan en un único mosaico georreferenciado utilizando la función `merge` de `rasterio`. Este proceso:

1. Abre cada archivo de predicción generado anteriormente.
2. Combina los parches en una sola matriz raster, calculando la transformada espacial correspondiente.
3. Actualiza la metadata (tamaño, transformada, tipo de dato) y escribe el resultado en un único archivo `pred_ccpp_test.tif`.

El mosaico resultante permite visualizar en un solo GeoTIFF la delimitación completa de las clases (fondo, urbano, rural) sobre toda el área cubierta por el conjunto de prueba.


In [ ]:
import rasterio
from rasterio.merge import merge
import glob
import os

In [ ]:
PRED_DIR = "/content/predictions"   # Ajustar si es necesario
pred_tiles = sorted(glob.glob(os.path.join(PRED_DIR, "*.tif")))
len(pred_tiles)

479

In [ ]:
src_files = [rasterio.open(p) for p in pred_tiles]

In [ ]:
mosaic, out_transform = merge(src_files)

In [ ]:
out_meta = src_files[0].meta.copy()
out_meta.update({
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_transform,
    "count": 1,
    "dtype": "uint8"
})

In [ ]:
out_final = "/content/pred_ccpp_test.tif"

with rasterio.open(out_final, "w", **out_meta) as dst:
    dst.write(mosaic)

In [ ]:
for src in src_files:
    src.close()